# Introduction/Business Problem

New York City and the city of Toronto are very diverse and are the financial capitals of their respective countries. Since we explored certain boroughs of  New York City and the city of Toronto and segmented and clustered their neighborhoods, lets try to cluster whole city and their entire neighborhoods to find similarities among them.

The aim of this project is to build a guide which will show which neighborhoods are similar between these two cities. This can be helpful for businneses that are trying to expand from one city to another or for people trying to relocate from one city to another and doesnt want to miss out the amenities thay had in the previous city. 

#Data

##New york data

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


New york has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 


In [ ]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


#### Load and explore the data


In [ ]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [ ]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

In [ ]:
#let's define a new variable that includes this data  in the features key, which is basically a list of the neighborhoods.
neighborhoods_data = newyork_data['features']

In [ ]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

#### Tranform the data into a _pandas_ dataframe


In [ ]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [ ]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.


In [ ]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Then let's loop through the data and fill the dataframe one row at a time.


In [ ]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [ ]:
#Lets check that the dataset has all 5 boroughs and 306 neighborhoods.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [ ]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.


In [ ]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

##Toronto data

###Web Scraping

Toronto has a total of 10 boroughs and 103 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 10 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

We will be using the following source to acquire the data about neighbourhoods in Toronto

Source: [https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [ ]:
url ="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
df = pd.read_html(url,header=0)
df

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

In [ ]:
#no of tables in  the source page
len(df)

3

In [ ]:
#so there are 3 tables, lets select the 1st table
df = df[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [ ]:
#dropping cells with a borough that is Not assigned.
df.drop(df.index[df['Borough'] == "Not assigned"], inplace = True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [ ]:
#assign nan to not assigned, if it exists in Neighbourhood
import numpy as np
df.replace("Not assigned",np.nan,inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [ ]:
#checking for null values
df.isnull().sum()

Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64

Hence there are no null values

In [ ]:
df.reset_index(drop=True,inplace=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [ ]:
df.shape

(103, 3)

Source of geographical coordinates of the neighborhoods in toronto:
http://cocl.us/Geospatial_data

In [ ]:
!wget -q http://cocl.us/Geospatial_data

In [ ]:
df1 = pd.read_csv("/content/Geospatial_data")
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
# merge df with df1 to add latitude/longitude for each neighborhood
df_merge = df.join(df1.set_index("Postal Code"),on="Postal Code")
df_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
#lets rename our dataframe
neighborhoods1 = df_merge

In [ ]:
#lets drop postal code column and rename neighbourhood to neighborhood
df_merge.drop("Postal Code",axis=1,inplace=True)

In [ ]:
df_merge.rename(columns={"Neighbourhood":"Neighborhood"},inplace=True)

In [ ]:
df_merge.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
#Examining number of boroughs and neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods1['Borough'].unique()),
        neighborhoods1.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto


In [ ]:
address = 'City of Toronto, Ontario ,Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of City of Toronto are 43.6534817, -79.3839347.


#### Create a map of City of Toronto with neighborhoods superimposed on top.


In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods1['Latitude'], neighborhoods1['Longitude'], neighborhoods1['Borough'], neighborhoods1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
#boroughs in toronto
neighborhoods1['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

##Combining both data and creating a new data frame

In [ ]:
#New york neighborhoods
neighborhoods.head(2)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939


In [ ]:
neighborhoods.shape

(306, 4)

In [ ]:
#Toronto neighborhoods
df_merge.head(2)

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572


In [ ]:
df_merge.shape

(103, 4)

In [ ]:
#Combined neighborhoods data
neighborhoods =neighborhoods.append(df_merge)

In [ ]:
neighborhoods.shape

(409, 4)

In [ ]:
neighborhoods.reset_index(drop=True,inplace=True)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [ ]:
neighborhoods['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island',
       'North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [ ]:
# create visualize both data
map_combined_cities = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_combined_cities)  
    
map_combined_cities

We can zoom into whichever city we are intersted in and see the neighborhoods

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [ ]:
CLIENT_ID = 'VN313PRPAGZCC510RQQTBYUPMGMLUUHEXXZ0LGWYBU2440H1' # your Foursquare ID
CLIENT_SECRET = 'YB12DGTPUO2UM2YP4CAYC1HNB4NOIWLAYM1NYE3HIPBUADYC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VN313PRPAGZCC510RQQTBYUPMGMLUUHEXXZ0LGWYBU2440H1
CLIENT_SECRET:YB12DGTPUO2UM2YP4CAYC1HNB4NOIWLAYM1NYE3HIPBUADYC


#### Let's create a function to get all venues in the neighborhoods 


In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
venues_list = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [ ]:
print(venues_list.shape)
venues_list.head()

(12275, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Let's check how many venues were returned for each neighborhood


In [ ]:
venues_list.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
Allerton,30,30,30,30,30,30
Annadale,9,9,9,9,9,9
Arden Heights,4,4,4,4,4,4
Arlington,5,5,5,5,5,5
Arrochar,22,22,22,22,22,22
Arverne,20,20,20,20,20,20
Astoria,96,96,96,96,96,96


In [ ]:
venues_list.shape

(12275, 7)

#### Let's find out how many unique categories can be curated from all the returned venues


In [ ]:
print('There are {} uniques categories.'.format(len(venues_list['Venue Category'].unique())))

There are 464 uniques categories.


<a id='item3'></a>


## Analyze Each Neighborhood


In [ ]:
# one hot encoding
combined_cities_onehot = pd.get_dummies(venues_list[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
combined_cities_onehot['Neighborhood'] = venues_list['Neighborhood'] 

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern R

In [ ]:
combined_cities_onehot.shape

(12275, 464)

In [ ]:
combined_cities_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12275 entries, 0 to 12274
Columns: 464 entries, Women's Store to Wings Joint
dtypes: object(1), uint8(463)
memory usage: 5.5+ MB


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [ ]:
combined_cities_grouped = combined_cities_onehot.groupby('Neighborhood').mean().reset_index()
combined_cities_grouped

In [ ]:
combined_cities_grouped.shape

(394, 464)

#### Let's print each neighborhood along with the top 5 most common venues


In [ ]:
num_top_venues = 5

for hood in combined_cities_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = combined_cities_grouped[combined_cities_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                 Print Shop  0.25
3                     Lounge  0.25
4              Women's Store  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.22
1         Coffee Shop  0.11
2        Skating Rink  0.11
3                 Gym  0.11
4  Athletics & Sports  0.11


----Allerton----
                venue  freq
0         Pizza Place  0.13
1       Deli / Bodega  0.10
2         Supermarket  0.10
3  Chinese Restaurant  0.07
4                 Spa  0.07


----Annadale----
            venue  freq
0     Pizza Place  0.22
1  Cosmetics Shop  0.11
2            Food  0.11
3    Liquor Store  0.11
4   Train Station  0.11


----Arden Heights----
           venue  freq
0       Pharmacy  0.25
1  Deli / Bodega  0.25
2    Coffee Shop  0.25
3    Pizza Place  0.25
4  Women's Store  0.00


----Arlington----
           venue  freq
0       Bus Stop

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = combined_cities_grouped['Neighborhood']

for ind in np.arange(combined_cities_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(combined_cities_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Print Shop,Breakfast Spot,Latin American Restaurant,Lounge,Wings Joint,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Dance Studio,Pub,Gym,Skating Rink,Sandwich Place,Coffee Shop,Event Service,Ethiopian Restaurant
2,Allerton,Pizza Place,Supermarket,Deli / Bodega,Spa,Chinese Restaurant,Intersection,Electronics Store,Grocery Store,Breakfast Spot,Check Cashing Service
3,Annadale,Pizza Place,Pub,Diner,Restaurant,Train Station,Food,Cosmetics Shop,Liquor Store,Duty-free Shop,Eastern European Restaurant
4,Arden Heights,Coffee Shop,Pharmacy,Deli / Bodega,Pizza Place,Wings Joint,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room


In [ ]:
neighborhoods_venues_sorted.shape

(394, 11)

<a id='item4'></a>


## Cluster Neighborhoods


Lets run _k_-means to cluster the neighborhood into 10 clusters, so that we can have more choice in identifying clusters that are very similar


In [ ]:
# set number of clusters
kclusters = 10

combined_cities_grouped_clustering = combined_cities_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(combined_cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 1, 1, 0, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
neighborhoods_venues_sorted = neighborhoods_venues_sorted.join(neighborhoods.set_index('Neighborhood'), on='Neighborhood')

neighborhoods_venues_sorted.head() # check the last columns!

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
0,0,Agincourt,Print Shop,Breakfast Spot,Latin American Restaurant,Lounge,Wings Joint,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Scarborough,43.794200,-79.262029
1,1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Dance Studio,Pub,Gym,Skating Rink,Sandwich Place,Coffee Shop,Event Service,Ethiopian Restaurant,Etobicoke,43.602414,-79.543484
2,1,Allerton,Pizza Place,Supermarket,Deli / Bodega,Spa,Chinese Restaurant,Intersection,Electronics Store,Grocery Store,Breakfast Spot,Check Cashing Service,Bronx,40.865788,-73.859319
3,1,Annadale,Pizza Place,Pub,Diner,Restaurant,Train Station,Food,Cosmetics Shop,Liquor Store,Duty-free Shop,Eastern European Restaurant,Staten Island,40.538114,-74.178549
4,1,Arden Heights,Coffee Shop,Pharmacy,Deli / Bodega,Pizza Place,Wings Joint,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Staten Island,40.549286,-74.185887


In [ ]:
neighborhoods_venues_sorted.shape

(403, 15)

Finally, let's visualize the resulting clusters


In [ ]:
neighborhoods_venues_sorted['Cluster Labels'].unique()

array([0, 1, 8, 5, 6, 3, 4, 2, 7, 9], dtype=int32)

In [ ]:
neighborhoods_venues_sorted['Cluster Labels'].value_counts()

0    167
1    160
5     57
6     10
9      2
8      2
4      2
7      1
3      1
2      1
Name: Cluster Labels, dtype: int64

>As we can seethere are two rows that are labeled nan, these can be outliers as they cannot be assigned to any group

In [ ]:
#removing rows with nan values
north_york_merged.dropna(axis=0,inplace=True)

In [ ]:
print("labels =", north_york_merged['Cluster Labels'].unique())
north_york_merged['Cluster Labels'].value_counts()

labels = [3. 0. 4. 1. 2.]


0.0    18
2.0     1
1.0     1
4.0     1
3.0     1
Name: Cluster Labels, dtype: int64

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Latitude'], neighborhoods_venues_sorted['Longitude'], neighborhoods_venues_sorted['Neighborhood'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


##Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 0, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Print Shop,Breakfast Spot,Latin American Restaurant,Lounge,Wings Joint,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service
7,Arverne,Surf Spot,Sandwich Place,Metro Station,Board Shop,Restaurant,Café,Beach,Bus Stop,Donut Shop,Burrito Place
8,Astoria,Bar,Middle Eastern Restaurant,Seafood Restaurant,Bakery,Greek Restaurant,Mediterranean Restaurant,Hookah Bar,Café,Indian Restaurant,Gym / Fitness Center
10,Auburndale,Italian Restaurant,Discount Store,Athletics & Sports,Toy / Game Store,Pet Store,Pharmacy,Supermarket,Bar,Noodle House,Fast Food Restaurant
13,Battery Park City,Park,Hotel,Coffee Shop,Boat or Ferry,Clothing Store,Gym,Memorial Site,Food Court,Burger Joint,Sandwich Place
14,Bay Ridge,Pizza Place,Italian Restaurant,Spa,American Restaurant,Bagel Shop,Bar,Grocery Store,Greek Restaurant,Cosmetics Shop,Pharmacy
15,Bay Terrace,Clothing Store,Donut Shop,Supermarket,Women's Store,American Restaurant,Shoe Store,Men's Store,Kids Store,Mobile Phone Shop,Cosmetics Shop
15,Bay Terrace,Clothing Store,Donut Shop,Supermarket,Women's Store,American Restaurant,Shoe Store,Men's Store,Kids Store,Mobile Phone Shop,Cosmetics Shop
17,Bayside,Bar,Pub,Sushi Restaurant,Indian Restaurant,Pizza Place,Mexican Restaurant,Bakery,Donut Shop,Italian Restaurant,Steakhouse
19,Bayview Village,Chinese Restaurant,Japanese Restaurant,Bank,Café,Wings Joint,Farm,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant


#### Cluster 2


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 1, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Dance Studio,Pub,Gym,Skating Rink,Sandwich Place,Coffee Shop,Event Service,Ethiopian Restaurant
2,Allerton,Pizza Place,Supermarket,Deli / Bodega,Spa,Chinese Restaurant,Intersection,Electronics Store,Grocery Store,Breakfast Spot,Check Cashing Service
3,Annadale,Pizza Place,Pub,Diner,Restaurant,Train Station,Food,Cosmetics Shop,Liquor Store,Duty-free Shop,Eastern European Restaurant
4,Arden Heights,Coffee Shop,Pharmacy,Deli / Bodega,Pizza Place,Wings Joint,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room
5,Arlington,Bus Stop,Boat or Ferry,Grocery Store,Deli / Bodega,Coffee Shop,Wings Joint,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
6,Arrochar,Pizza Place,Bagel Shop,Bus Stop,Italian Restaurant,Deli / Bodega,Liquor Store,Mediterranean Restaurant,Supermarket,Middle Eastern Restaurant,Taco Place
9,Astoria Heights,Italian Restaurant,Bowling Alley,Pizza Place,Burger Joint,Bus Station,Supermarket,Bakery,Chinese Restaurant,Museum,Cocktail Bar
11,Bath Beach,Pizza Place,Chinese Restaurant,Bubble Tea Shop,Fast Food Restaurant,Donut Shop,Cantonese Restaurant,Gas Station,Italian Restaurant,Pharmacy,Ice Cream Shop
12,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Grocery Store,Mobile Phone Shop,Bridal Shop,Sandwich Place,Shopping Mall,Gas Station,Sushi Restaurant,Middle Eastern Restaurant
16,Baychester,Donut Shop,Cosmetics Shop,Mattress Store,Sandwich Place,Gym / Fitness Center,Food & Drink Shop,Bank,Electronics Store,Men's Store,Discount Store


#### Cluster 3


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 2, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
303,Roselawn,Garden,Wings Joint,Farmers Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service


#### Cluster 4


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 3, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
175,"Humberlea, Emery",Baseball Field,Wings Joint,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service


#### Cluster 5


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 4, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
183,Jamaica Estates,Intersection,Dog Run,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service
228,"Milliken, Agincourt North, Steeles East, L'Amo...",Intersection,Park,Playground,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant


#### Cluster 6


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 5, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Belle Harbor,Beach,Pub,Spa,Deli / Bodega,Bakery,Chinese Restaurant,Bagel Shop,Donut Shop,Mexican Restaurant,Boutique
30,Bergen Beach,Harbor / Marina,Park,Athletics & Sports,Baseball Field,Playground,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service
33,Bloomfield,Theme Park,Bus Stop,Recreation Center,Burger Joint,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room
36,Breezy Point,Beach,Monument / Landmark,Trail,Bus Stop,Wings Joint,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service
45,Brownsville,Moving Target,Fried Chicken Joint,Trail,Plaza,Playground,Pizza Place,Performing Arts Venue,Chinese Restaurant,Park,Farmers Market
49,Butler Manor,Pool,Baseball Field,Gas Station,Wings Joint,Farmers Market,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
86,Country Club,Sandwich Place,Athletics & Sports,Playground,Trail,Wings Joint,Factory,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
90,Davisville North,Food & Drink Shop,Dance Studio,Gym / Fitness Center,Sandwich Place,Department Store,Hotel,Park,Breakfast Spot,Fish & Chips Shop,Film Studio
97,Downsview,Grocery Store,Park,Business Service,Hotel,Shopping Mall,Gym / Fitness Center,Baseball Field,Bank,Athletics & Sports,Korean Restaurant
97,Downsview,Grocery Store,Park,Business Service,Hotel,Shopping Mall,Gym / Fitness Center,Baseball Field,Bank,Athletics & Sports,Korean Restaurant


#### Cluster 7


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 6, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Caledonia-Fairbanks,Park,Women's Store,Pool,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
72,Clason Point,Park,Pool,Boat or Ferry,South American Restaurant,Bus Stop,Grocery Store,Convenience Store,Curling Ice,Empanada Restaurant,Cupcake Shop
106,"East Toronto, Broadview North (Old East York)",Intersection,Park,Convenience Store,Accessories Store,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
195,Lawrence Park,Park,Bus Line,Swim School,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service
269,Parkwoods,Park,Food & Drink Shop,Wings Joint,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
318,Somerville,Park,Wings Joint,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service
346,"The Kingsway, Montgomery Road, Old Mill North",Park,River,Wings Joint,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
349,Todt Hill,Park,Wings Joint,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service
376,Weston,Convenience Store,Park,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service
392,York Mills West,Convenience Store,Park,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service


#### Cluster 8


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 7, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
305,"Rouge Hill, Port Union, Highland Creek",Bar,Wings Joint,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service


#### Cluster 9


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 8, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Bayswater,Tennis Court,Playground,Wings Joint,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
311,Scarborough Village,Playground,Wings Joint,Farmers Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service


#### Cluster 10


In [ ]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 9, neighborhoods_venues_sorted.columns[[1] + list(range(2,12))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
370,"West Deane Park, Princess Gardens, Martin Grov...",Middle Eastern Restaurant,Farmers Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service,Event Space
377,"Wexford, Maryvale",Auto Garage,Bakery,Sandwich Place,Middle Eastern Restaurant,Wings Joint,Farm,Electronics Store,Empanada Restaurant,Entertainment Service,Escape Room


In [ ]:
#Number of neihbor hoods in each clusters
neighborhoods_venues_sorted['Cluster Labels'].value_counts()

0    167
1    160
5     57
6     10
9      2
8      2
4      2
7      1
3      1
2      1
Name: Cluster Labels, dtype: int64

## Results and discussion

- Clusters 3,4,8 are unique to their city as they have only one neighborhood
- Clusters 5,9 has two neighborhoods,each one belonging to different cities.
- Cluster 10 has two neighborhoods both belonging to Toronto
- Cluster 7 has 10 neighborhoods, of which some belong to New York city and the rest belonging to Toronto city  
- Cluster 6 has 57 neighborhoods, of which some belong to New York city and the rest belonging to Toronto city 
- Cluster 2 has 160 neighborhoods, of which some belong to New York city and the rest belonging to Toronto city 
- Cluster 1 has 167 neighborhoods(the maximum), of which some belong to New York city and the rest belonging to Toronto city 

As we can see most of the neighbourhoods fall into cluster 1 and 2 due to the high frequency of venues like coffee shops,cafe,gym, parks, bakery, lounges etc.

These clusters can be used by the people relocating from one city to another city, to find the neighborhoobs with similar amenities to their previous city.

They can be also used for creating business oppurtunities like finding clusters with low frequency of a certain venue and high polulation density. 


